# Glucose

This notebook is to be used to examine blood glucose record entries in the Apple Health export.

In [ ]:

from apple_health_analysis.healthdata import get_healthdata

print("Getting HealthData -- this may take a while...")
healthdata = get_healthdata(use_cache=True)
print("Done getting HealthData")

In [ ]:
glucose_records = healthdata.records[
    healthdata.records.record_type == "HKQuantityTypeIdentifierBloodGlucose"
]
print(
    f"{len(glucose_records)} glucose records found out of {len(healthdata.records)} records total"
)

In [ ]:
print(glucose_records.unit.describe())

In [ ]:
print(glucose_records.value.describe())

In [ ]:
import seaborn as sns

sns.set_theme(style="darkgrid")

sns.lineplot(
    x="start_date",
    y="value",
    data=glucose_records,
)